In [17]:
from sklearn import datasets

newsgroups = datasets.fetch_20newsgroups(
                    subset='all', 
                    categories=['alt.atheism', 'sci.space']
             )
#newsgroups.data[1]
#type(newsgroups.data)
len(newsgroups.data)

1786

In [100]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

vectorizer = TfidfVectorizer()
vect_feat = vectorizer.fit_transform(iter(newsgroups.data))

print(type(vectorizer.idf_))
print(vectorizer.idf_.size)

print(type(vect_feat))
print(vect_feat.shape)

feature_mapping = vectorizer.get_feature_names()
print(len(feature_mapping))
print(feature_mapping[1:10])


<class 'numpy.ndarray'>
28382
<class 'scipy.sparse.csr.csr_matrix'>
(1786, 28382)
28382
['000', '0000', '00000', '000000', '000021', '000050', '000062david42', '000406', '000410']


In [103]:
from sklearn import svm
from sklearn.cross_validation import KFold
from sklearn.grid_search import GridSearchCV

X = vect_feat
y = newsgroups.target

grid = {'C': np.power(10.0, np.arange(-5, 5))}
cv = KFold(y.size, n_folds=5, shuffle=True, random_state=241)
clf = svm.SVC(kernel='linear', random_state=241)
gs = GridSearchCV(clf, grid, scoring='accuracy', cv=cv)
gs.fit(X, y)

GridSearchCV(cv=sklearn.cross_validation.KFold(n=1786, n_folds=5, shuffle=True, random_state=241),
       error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=241, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': array([  1.00000e-05,   1.00000e-04,   1.00000e-03,   1.00000e-02,
         1.00000e-01,   1.00000e+00,   1.00000e+01,   1.00000e+02,
         1.00000e+03,   1.00000e+04])},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [107]:
print(type(gs.grid_scores_))
print(len(gs.grid_scores_))

for a in gs.grid_scores_:
    print(a.mean_validation_score) # — оценка качества по кросс-валидации
    print(a.parameters) # — значения параметров

<class 'list'>
10
0.552631578947
{'C': 1.0000000000000001e-05}
0.552631578947
{'C': 0.0001}
0.552631578947
{'C': 0.001}
0.552631578947
{'C': 0.01}
0.950167973124
{'C': 0.10000000000000001}
0.993281075028
{'C': 1.0}
0.993281075028
{'C': 10.0}
0.993281075028
{'C': 100.0}
0.993281075028
{'C': 1000.0}
0.993281075028
{'C': 10000.0}


In [108]:
clf_new = svm.SVC(kernel='linear', C = 10000, random_state=241)
clf_new.fit(X, y)

SVC(C=10000, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=241, shrinking=True,
  tol=0.001, verbose=False)

In [117]:
print(type(clf_new.coef_))
print(clf_new.coef_.shape)

print(type(clf_new.support_vectors_))
print(clf_new.support_vectors_.shape)

<class 'scipy.sparse.csr.csr_matrix'>
(1, 28382)
<class 'scipy.sparse.csr.csr_matrix'>
(781, 28382)


In [263]:
print(np.expand_dims(np.array(feature_mapping),axis=1).shape)
print(np.transpose(clf_new.coef_.toarray()).shape)
feature_coef = np.concatenate((np.expand_dims(np.array(feature_mapping),axis=1),np.transpose(clf_new.coef_.toarray())), axis=1)
print(feature_coef)
print(type(feature_coef))
print(feature_coef.shape)
feature_coef[0:9,:]

(28382, 1)
(28382, 1)
[['00' '0.2736578152866335']
 ['000' '-0.10705496994648023']
 ['0000' '0.0']
 ..., 
 ['zwarte' '0.01994628492375098']
 ['zwork' '0.05950163593467093']
 ['zyklon' '-0.0030138075340242526']]
<class 'numpy.ndarray'>
(28382, 2)


array([['00', '0.2736578152866335'],
       ['000', '-0.10705496994648023'],
       ['0000', '0.0'],
       ['00000', '0.0'],
       ['000000', '0.0'],
       ['000021', '0.004948641987602272'],
       ['000050', '0.0'],
       ['000062david42', '0.004526452691232835'],
       ['000406', '-0.06300157236227182']], 
      dtype='<U80')

In [205]:
#feature_coef_sorted = np.sort(feature_coef, axis = 0)
#feature_coef_sorted = feature_coef_sorted[::-1]
#print(feature_coef_sorted[0:19])

In [296]:
import pandas as pd

feature_coef_mod = pd.DataFrame(feature_coef)
feature_coef_mod.iloc[:,1] = feature_coef_mod.iloc[:,1].astype(np.float64).abs()
feature_coef_mod.sort_values(1, ascending=False, inplace=True)
feature_coef_mod
#type(feature_coef_mod[2,1].astype(np.float64))
#print(np.absolute(feature_coef_mod[:,1].astype(np.float64)))
#feature_coef_mod[:,1] = feature_coef_mod[:,1].astype(np.float64)
#type(feature_coef_mod[2,1])
#feature_coef_mod[:,1] = np.absolute(feature_coef_mod[:,1])
#print(feature_coef_mod[0:9,:])
#feature_coef_sorted = feature_coef_mod[feature_coef_mod[:,1].argsort()]
#feature_coef_sorted = feature_coef_sorted[::-1]
#print(feature_coef_sorted[0:19])

,0,1
24019,space,2.720304
12871,god,1.971924
5088,atheism,1.299935
5093,atheists,1.257272
17802,moon,1.248378
23673,sky,1.195954
21850,religion,1.155796
5776,bible,1.126725
15606,keith,1.107114
18430,nick,1.089924
